In [1]:
'''
!pip install datasets

!pip install transformers 
!pip install sentencepiece

!pip install sacrebleu
'''

'\n!pip install datasets\n\n!pip install transformers \n!pip install sentencepiece\n\n!pip install sacrebleu\n'

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from torch.nn.parallel import DataParallel
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
model = DataParallel(model)


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
new_tokens = ['<H>', '<R>', '<T>']
new_tokens_vocab = {}
new_tokens_vocab['additional_special_tokens'] = []
for idx, t in enumerate(new_tokens):
    new_tokens_vocab['additional_special_tokens'].append(t)
num_added_toks = tokenizer.add_special_tokens(new_tokens_vocab)

tokenizer.add_tokens("[MASK]")
tokenizer.mask_token = "[MASK]"
tokenizer.mask_token_id = tokenizer.convert_tokens_to_ids("[MASK]")

In [4]:
class WebNLGDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.prefix = "translate from Graph to Text: "

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        # preprocess the input graph
        try:
            triples = item['original_triple_sets']['otriple_set']
            input_text = self.prefix
            for outer_list in triples: 
                for triple in outer_list:
                    triple_txt = triple.split("|")
                    input_text += " <H> " + triple_txt[0] + " <R> " + triple_txt[1] + " <T> " + triple_txt[2]
        except (KeyError, IndexError):
            print("1")
            print(item['original_triple_sets']['otriple_set'])
            print(item['original_triple_sets']['otriple_set'][0])
            print(triples)
            input_text = self.prefix
        # preprocess the target text
        try:
            target_text = item['lex']['text'][0]
        except (KeyError, IndexError):
            print("2")
            print(item)
            #print(item['original_triple_sets']['otriple_set'])
            target_text = ""
        #print(item)
        #print(input_text)
        # encode the inputs and targets using the tokenizer
        input_ids = tokenizer.encode(input_text, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
        target_ids = tokenizer.encode(target_text, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
        return input_ids.squeeze(0), target_ids.squeeze(0)


In [5]:
class DartDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.prefix = "translate from Graph to Text: "

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        # preprocess the input graph
        try:
            tripleset = item['tripleset']
            input_text = self.prefix
            for triple in tripleset: 
                triple_txt = triple
                input_text += " <H> " + triple_txt[0] + " <R> " + triple_txt[1] + " <T> " + triple_txt[2]
        except (KeyError, IndexError):
            print("1")
            print(item['tripleset'])
            input_text = self.prefix
        # preprocess the target text
        try:
            target_text = item['annotations']['text'][0]
        except (KeyError, IndexError):
            print("2")
            print(item)
            target_text = ""
        # encode the inputs and targets using the tokenizer
        input_ids = tokenizer.encode(input_text, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
        target_ids = tokenizer.encode(target_text, return_tensors='pt', padding='max_length', max_length=128, truncation=True)

        #print decoding of input and target
        #print(tokenizer.decode(input_ids[0], skip_special_tokens=True))
        #print(tokenizer.decode(target_ids[0], skip_special_tokens=True))
        return input_ids.squeeze(0), target_ids.squeeze(0)


In [6]:
class E2ENLGDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.prefix = "Translate from Meaning Representation to Text: "

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        # preprocess the input meaning representation
        try:
            input_text = self.prefix + item['meaning_representation']
        except KeyError:
            print("Failed to access 'meaning_representation' for item at index: ", idx)
            input_text = self.prefix

        # preprocess the target text
        try:
            target_text = item['human_reference']
        except KeyError:
            print("Failed to access 'human_reference' for item at index: ", idx)
            target_text = ""

        # encode the inputs and targets using the tokenizer
        input_ids = tokenizer.encode(input_text, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
        target_ids = tokenizer.encode(target_text, return_tensors='pt', padding='max_length', max_length=128, truncation=True)

        # print decoded input and target texts
        #print("Input Text: ", tokenizer.decode(input_ids[0], skip_special_tokens=True))
        #print("Target Text: ", tokenizer.decode(target_ids[0], skip_special_tokens=True))

        return input_ids.squeeze(0), target_ids.squeeze(0)


In [7]:
class BalancedCombinedDataset(Dataset):
    def __init__(self, dataset1, dataset2, dataset3):
        self.datasets = [dataset1, dataset2, dataset3]
        self.lengths = [len(d) for d in self.datasets]
        self.max_length = max(self.lengths)

    def __len__(self):
        return self.max_length * len(self.datasets)

    def __getitem__(self, idx):
        dataset_idx = idx % len(self.datasets)
        item_idx = idx // len(self.datasets)
        item_idx = item_idx % self.lengths[dataset_idx]  # repeat smaller datasets
        return self.datasets[dataset_idx][item_idx]


In [8]:
MAX_INPUT_LENGTH = 128
MAX_TARGET_LENGTH = 128
tokenizer.model_max_length = MAX_INPUT_LENGTH
model.module.config.max_length = MAX_TARGET_LENGTH

# set up the device (GPU or CPU)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

DataParallel(
  (module): T5ForConditionalGeneration(
    (shared): Embedding(32128, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseGatedActDense(
                (wi_0): Linear(in_features=768, out_features=2048, bias=False)
                (wi_1): Linear

In [9]:
# load the WebNLG dataset
webnlg_data = load_dataset('web_nlg', 'release_v3.0_en')['train']
dart_data = load_dataset('dart', split='train', ignore_verifications=True)
e2enlg_data = load_dataset('e2e_nlg', split='train')

webnlg_data_val = load_dataset('web_nlg', 'release_v3.0_en')['dev']
dart_data_val = load_dataset('dart', split='validation', ignore_verifications=True)
e2enlg_data_val = load_dataset('e2e_nlg', split='validation')

Generating train split:   0%|          | 0/13211 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1667 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5713 [00:00<?, ? examples/s]

Dataset web_nlg downloaded and prepared to /root/.cache/huggingface/datasets/web_nlg/release_v3.0_en/0.0.0/28ffb892f7f42450dd9558684aa43bcaf44b1b3bf0d77cb8d73534646af88dda. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

...

Generating train split:   0%|          | 0/42061 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4672 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4693 [00:00<?, ? examples/s]

Dataset e2e_nlg downloaded and prepared to /root/.cache/huggingface/datasets/e2e_nlg/default/0.0.0/bfeceb720929c2705bd227d1cfe5eaaab102a0bdac10dad618dac1e00c737430. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
webnlg_dataset = WebNLGDataset(webnlg_data)
dart_dataset = DartDataset(dart_data)
e2enlg_dataset = E2ENLGDataset(e2enlg_data)

train_data = BalancedCombinedDataset(webnlg_dataset, dart_dataset, e2enlg_dataset)

webnlg_dataset_val = WebNLGDataset(webnlg_data_val)
dart_dataset_val = DartDataset(dart_data_val)
e2enlg_dataset_val = E2ENLGDataset(e2enlg_data_val)


val_data = BalancedCombinedDataset(webnlg_dataset_val, dart_dataset_val, e2enlg_dataset_val)

# Adaptive pretraining

For STA, we fine-tuned the PLMs on a small amount of labeled data from the target task using a maximum likelihood estimation (MLE) objective. This involves training the model to maximize the likelihood of generating the correct output given the input graph and labeled data. This process helps to further adapt the PLM to the specific requirements of the target task and improve its performance on that task.

In [6]:
import random

pretrain_texts = []
for sample in dataset:
    try:
        text = sample['lex']['text'][0]
        pretrain_texts.append(text)
    except (KeyError, IndexError):
        continue

tokenized_inputs = tokenizer(pretrain_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')
input_ids = tokenized_inputs['input_ids']
attention_mask = tokenized_inputs['attention_mask']

pretrain_data = torch.utils.data.TensorDataset(input_ids, attention_mask)

pretrain_loader = torch.utils.data.DataLoader(pretrain_data, batch_size=int(60), shuffle=True)

pretrain_optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
pretrain_criterion = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

pretrain_epochs = 2  # Set the number of pre-training epochs
masking_prob = 0.15  # Probability of masking a token

if tokenizer.mask_token is None:
    # Manually set a mask token if not already defined
    tokenizer.add_tokens("[MASK]")
    tokenizer.mask_token = "[MASK]"
    tokenizer.mask_token_id = tokenizer.convert_tokens_to_ids("[MASK]")

for epoch in range(pretrain_epochs):
    running_loss = 0.0
    for inputs, attention_mask in pretrain_loader:
        inputs = inputs.to(device)
        attention_mask = attention_mask.to(device)
        batch_size, seq_length = inputs.shape
        
        # Create a mask for randomly selected tokens
        mask = torch.rand(inputs.shape) < masking_prob
        
        # Randomly replace selected tokens with [MASK] token
        masked_inputs = inputs.clone()
        masked_inputs[mask] = tokenizer.mask_token_id
        
        pretrain_optimizer.zero_grad()
        outputs = model(input_ids=masked_inputs, attention_mask=attention_mask, decoder_input_ids=inputs)
        
        # Compute the loss only for the masked tokens
        masked_logits = outputs.logits[mask]
        masked_labels = inputs[mask]
        loss = pretrain_criterion(masked_logits.view(-1, masked_logits.size(-1)), masked_labels.view(-1))
        
        loss.backward()
        pretrain_optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(pretrain_data)
    print(f"Pretrain Epoch {epoch+1}/{pretrain_epochs} - loss: {epoch_loss:.4f}")


Pretrain Epoch 1/2 - loss: 2.0408
Pretrain Epoch 2/2 - loss: 0.1616


For LMA, we first fine-tuned the PLMs on a small amount of task-specific data using a masked language modeling objective. This involves randomly masking some tokens in the input sequence and training the model to predict the masked tokens based on the context provided by the unmasked tokens. This process helps to adapt the PLM to the specific characteristics of the target task and improve its performance on that task.

# Finetuning

In [11]:
from torch.utils.data import DataLoader
from tqdm import tqdm

"""# Setting up the data loader
train_data = WebNLGDataset(dataset)
val_dataset = load_dataset('web_nlg', 'release_v3.0_en')['dev']
val_data = WebNLGDataset(val_dataset)  # You need to prepare the validation dataset similarly to your training dataset"""

batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)  # No need to shuffle the validation set

# Setting up the optimizer and the loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

# Initialize early stopping parameters
early_stopping_patience = 2  # Number of epochs with no improvement after which training will be stopped
no_improve_epochs = 0
min_val_loss = float('inf')

num_epochs = 1
for epoch in range(num_epochs):
    # Training
    model.train()
    running_loss = 0.0
    for inputs, targets in tqdm(train_loader):
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs, labels=targets)
        loss = criterion(outputs.logits.view(-1, outputs.logits.size(-1)), targets.view(-1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_data)
    print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {epoch_loss:.4f}")
    
    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs, labels=targets)
            loss = criterion(outputs.logits.view(-1, outputs.logits.size(-1)), targets.view(-1))
            val_loss += loss.item() * inputs.size(0)
    val_loss = val_loss / len(val_data)
    print(f"Epoch {epoch+1}/{num_epochs} - Validation loss: {val_loss:.4f}")
    
    # Check early stopping condition
    if val_loss < min_val_loss:
        min_val_loss = val_loss
        no_improve_epochs = 0
    else:
        no_improve_epochs += 1
        if no_improve_epochs >= early_stopping_patience:
            print("Early stopping invoked.")
            break


  0%|          | 0/3944 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
100%|██████████| 3944/3944 [1:24:51<00:00,  1.29s/it]


Epoch 1/1 - Train loss: 0.9621
Epoch 1/1 - Validation loss: 0.8109


In [12]:
# Save the entire model
torch.save(model, 'model_T5_flan_base_multi_1_epoch')
print("Model saved successfully.")

Model saved successfully.


In [105]:
# Load the model
#model = torch.load('/kaggle/input/models/model_T5_flan_small_88')

# Print a confirmation message
print("Model loaded successfully.")

Model loaded successfully.


## are we accounting for the multiple texts targets in the bleu? it doesn't look like it

In [13]:
!pip install sacrebleu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 3.8 MB/s eta 0:00:00


In [14]:
batch_size=32

In [15]:
from sacrebleu import corpus_bleu
from random import sample
from tqdm import tqdm


# load the WebNLG validation dataset
validation_dataset = load_dataset('web_nlg', 'release_v3.0_en')['test']
validation_dataset = [sample for sample in validation_dataset if sample['lex']['text']] # filter out samples with empty targets 
# Select a subset of the validation dataset
#subset_size = 10  # Choose the desired subset size
#validation_subset = sample(list(validation_dataset), subset_size)
validation_data = WebNLGDataset(validation_dataset)

# set up the validation data loader
validation_loader = DataLoader(validation_data, batch_size=batch_size, shuffle=False)

# switch model to evaluation mode
model.eval()

# generate predictions for the validation dataset
predictions = []
references = []
with torch.no_grad():
    for inputs, targets in tqdm(validation_loader, desc='Validation Progress', leave=False):
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model.module.generate(inputs, max_length=MAX_TARGET_LENGTH, num_beams=4)
        # convert token IDs to strings
        predicted_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        target_texts = tokenizer.batch_decode(targets, skip_special_tokens=True)
        # append predicted and target texts for BLEU evaluation
        predictions.extend(predicted_texts)
        references.extend(target_texts)


  0%|          | 0/3 [00:00<?, ?it/s]

Validation Progress:   0%|          | 0/123 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


In [16]:
# calculate BLEU scores
#bleu = corpus_bleu(predictions, [references])

multiple_references = []
for i in range(len(validation_dataset)):
    multiple_references.append(validation_dataset[i]['lex']['text'])
    
bleu = corpus_bleu(predictions, references)
bleu_multiple = corpus_bleu(predictions, multiple_references)

print(f"BLEU score: {bleu.score}")
print(f"BLEU score with multiple references: {bleu_multiple.score}")

BLEU score: 0.19767614950377796
BLEU score with multiple references: 94.80068745915912


In [17]:
# calculate BLEU scores
#bleu = corpus_bleu(predictions, [references])

multiple_references = []
for i in range(len(validation_dataset)):
    multiple_references.append(validation_dataset[i]['lex']['text'])
    
# First, determine the maximum length of sublists
max_len = max(len(refs) for refs in multiple_references)

# Then pad all sublists to that length
padded_references = [refs * (max_len // len(refs)) + refs[:max_len % len(refs)] for refs in multiple_references]
    
bleu = corpus_bleu(predictions, references)
bleu_multiple = corpus_bleu(predictions, padded_references)

print(f"BLEU score: {bleu.score}")
print(f"BLEU score with padded references: {bleu_multiple.score}")

BLEU score: 0.19767614950377796
BLEU score with padded references: 87.96615516145138


In [18]:
from datasets import load_metric

metric = load_metric('sacrebleu')

# First, determine the maximum length of sublists
max_len = max(len(refs) for refs in multiple_references)

# Then pad all sublists to that length
padded_references = [refs * (max_len // len(refs)) + refs[:max_len % len(refs)] for refs in multiple_references]

# Now 'padded_references' is a list of lists, where each sublist has the same length.
# We can now compute the SacreBLEU score.

# Note the change in the compute line
score = metric.compute(predictions=predictions, references = padded_references)

print(f"SacreBLEU score: {score['score']}")


SacreBLEU score: 37.65135126120528


In [19]:
from sacrebleu import corpus_chrf
# Calculate CHR F++ scores
chrf = corpus_chrf(predictions, [references])
chrf_multiple = corpus_chrf(predictions, multiple_references)
print(f"CHR F++ score: {chrf.score}")
print(f"CHR F++ score with multiple references: {chrf_multiple.score}")

CHR F++ score: 58.760976605324544
CHR F++ score with multiple references: 75.17566273466703


In [20]:
from sacrebleu import corpus_chrf
# Calculate CHR F++ scores
chrf = corpus_chrf(predictions, [references])
chrf_multiple = corpus_chrf(predictions, padded_references)
print(f"CHR F++ score: {chrf.score}")
print(f"CHR F++ score with multiple references: {chrf_multiple.score}")

CHR F++ score: 58.760976605324544
CHR F++ score with multiple references: 74.93981222366703


In [21]:
!pip install bert_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.8 MB/s eta 0:00:00


In [22]:
from datasets import load_metric
import numpy as np


metric = load_metric('bertscore')

assert len(predictions) == len(references), "The number of predictions and references should be the same."

# Compute the score
score = metric.compute(predictions=predictions, references=references, lang='en')

print(f"BERTScore: {np.mean(score['precision'])}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore: 0.9369189899071982


In [23]:
i=5
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print('----')
print(predictions[i])
print('----')
print(multiple_references[i])

[['Bootleg_Series_Volume_1:_The_Quine_Tapes | previousWork | Squeeze_(The_Velvet_Underground_album)', 'Squeeze_(The_Velvet_Underground_album) | subsequentWork | 1969:_The_Velvet_Underground_Live']]
----
The Bootleg Series Volume 1: The Quine Tapes was preceded by Squeeze (The Velvet Underground album) and followed by 1969: The Velvet Underground Live.
----
['The album 1969: The Velvet Underground Live is preceded by the Velvet Underground album Squeeze, which was followed by The Quine Tapes.', 'The Velvet Underground album Bootleg Series Volume 1: The Quine Tapes was preceded by the album Squeeze, which was followed by the live album 1969: The Velvet Underground Live.', 'The Bootleg Series Volume I: The Quine Tapes is preceded by the Velvet Underground album Squeeze which was itself followed by the album 1969: The Velvet Underground Live.']


In [24]:
i=10
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print('----')
print(predictions[i])
print('----')
print(multiple_references[i])

[['Piotr_Hallmann | birthDate | 1987-01-01'], ['Piotr_Hallmann | birthDate | 1987-08-25']]
----
Piotr Hallmann was born on the 25th of August 1987.
----
['Piotr Hallmann was born on August 25, 1987.', "Piotr Hallmann's birthday is August 25th 1987.", 'Piotr Hallmann was born on the 25th of August 1987.']


In [25]:
i=50
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print('----')
print(predictions[i])
print('----')
print(multiple_references[i])

[['Alan_Shepard | birthDate | "1923-11-18"^^xsd:date', 'Alan_Shepard | deathPlace | California', 'Alan_Shepard | birthPlace | New_Hampshire', 'Alan_Shepard | mission | Apollo_14']]
----
Alan Shepard was born in New Hampshire on November 18, 1923. He was a crew member of Apollo 14 and died in California.
----
['Alan Shepard was a crew member of Apollo 14 who was born November 18th, 1923, in New Hampshire and died in California.', 'Alan Shepard was born in New Hampshire on November 18, 1923. He was a crew member of Apollo 14, and died later on in California.', 'Alan Shepard was born on Nov 18, 1923 in New Hampshire, was a member of the Apollo 14 crew and died in California.']


In [26]:
i=0
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print('----')
print(predictions[i])
print('----')
print(multiple_references[i])

[['Estádio_Municipal_Coaracy_da_Mata_Fonseca | location | Arapiraca', 'Agremiação_Sportiva_Arapiraquense | league | Campeonato_Brasileiro_Série_C', 'Campeonato_Brasileiro_Série_C | country | Brazil', 'Agremiação_Sportiva_Arapiraquense | nickname | "\'\'Alvinegro"@en', 'Agremiação_Sportiva_Arapiraquense | ground | Estádio_Municipal_Coaracy_da_Mata_Fonseca']]
----
Agremiaço Sportiva Arapiraquense play in the Campeonato Brasileiro Série C league in Brazil. The Estádio Municipal Coaracy da Mata Fonseca is located in Arapiraca.
----
['Estádio Municipal Coaracy da Mata Fonseca is the name of the ground of Agremiação Sportiva Arapiraquense in Arapiraca. Agremiação Sportiva Arapiraquense, nicknamed "Alvinegro", lay in the Campeonato Brasileiro Série C league from Brazil.', 'Estádio Municipal Coaracy da Mata Fonseca is the name of the ground of Agremiação Sportiva Arapiraquense in Arapiraca. Alvinegro, the nickname of Agremiação Sportiva Arapiraquense, play in the Campeonato Brasileiro Série C 

In [27]:
i=70
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print('----')
print(predictions[i])
print('----')
print(multiple_references[i])

[['Pontiac_Rageous | assembly | Michigan', 'Pontiac_Rageous | assembly | Detroit', 'Pontiac_Rageous | productionEndYear | 1997-01-01']]
----
The Pontiac Rangeous was assembled in Detroit, Michigan and ended its production on January 1, 1997.
----
['The Pontiac Rageous assembled in Michigan with assembly line in Detroit was last produced in 1997.', 'Ending its production in 1997, the Pontiac Rageous was assembled in Detroit, Michigan.', 'Ending in 1997, the Pontiac Rageous was assembled in Detroit, Michigan.']


In [28]:
i=130
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print('----')
print(predictions[i])
print('----')
print(multiple_references[i])

[['McVeagh_of_the_South_Seas | director | Harry_Carey_(actor_born_1878)', 'McVeagh_of_the_South_Seas | writer | Harry_Carey_(actor_born_1878)', 'McVeagh_of_the_South_Seas | producer | The_Progressive_Motion_Picture_Company']]
----
The director of McVeagh of the South Seas was Harry Carey, who was born in 1888. The film was produced by The Progressive Motion Picture Company and was directed by Harry Carey.
----
['"McVeagh of the South Seas" was written and directed by Harry Carey born in 1878 and produced by Progressive Motion Picture Company.', 'Harry Carey, an actor born in 1878, was the director and writer for the film McVeagh of the South Seas which was produced by the Progressive Motion Picture Company.', 'Harry Carey, an actor and director, was born in 1878. He was the writer of the movie McVeagh of the South Seas, which was produced by the Progressive Motion Picture Company. As well as writing the film script, Carey also acted a role in the movie, and directed it.']


In [29]:
i=1861
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print('----')
print(predictions[i])
print('----')
print(multiple_references[i])

[['Akeem_Ayers | currentteam | "Los Angeles Rams"@en', 'Akeem_Ayers | debutTeam | Tennessee_Titans']]
----
Akeem Ayers made his debut for the Tennessee Titans and now plays for the Los Angeles Rams.
----
['Akeem Ayers made his debut for the Tennessee Titans and currently plays for the Los Angeles Rams.']


In [30]:
i=1860
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print('----')
print(predictions[i])
print('----')
print(multiple_references[i])

[['Turn_Me_On_(album) | runtime | 2106.0', 'Turn_Me_On_(album) | artist | The_Honeymoon_Killers_(American_band)', 'Turn_Me_On_(album) | genre | Punk_blues', 'Turn_Me_On_(album) | producer | The_Honeymoon_Killers_(American_band)', 'Turn_Me_On_(album) | genre | Noise_rock', 'Turn_Me_On_(album) | previousWork | Let_It_Breed']]
----
The album Turn Me On, produced by The Honeymoon Killers, is a punk blues album with a run time of 2106.0.
----
['Turn Me On by the Honeymoon Killers is a punk blues album in the noise rock genre. The run time is 35.1 minutes and was preceded by the Let it Breed album.']


In [32]:
dataset = load_dataset('web_nlg', 'release_v3.0_en')['test']
count_empty_text = 0
for sample in dataset:
    if not sample['lex']['text']:
        count_empty_text += 1

print(f"Number of samples with empty 'lex' 'text' field: {count_empty_text}")


  0%|          | 0/3 [00:00<?, ?it/s]

Number of samples with empty 'lex' 'text' field: 1779


----

### testing the other datasets

In [34]:
from sacrebleu import corpus_bleu
from random import sample
from tqdm import tqdm

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
batch_size = 32
MAX_INPUT_LENGTH = 128
MAX_TARGET_LENGTH = 128
tokenizer.model_max_length = MAX_INPUT_LENGTH
model.module.config.max_length = MAX_TARGET_LENGTH


# load the WebNLG validation dataset
validation_dataset = load_dataset('dart', split='test', ignore_verifications=True)
# Select a subset of the validation dataset
validation_data = DartDataset(validation_dataset)

# set up the validation data loader
validation_loader = DataLoader(validation_data, batch_size=batch_size, shuffle=False)

# switch model to evaluation mode
model.eval()

# generate predictions for the validation dataset
predictions = []
references = []
with torch.no_grad():
    for inputs, targets in tqdm(validation_loader, desc='Validation Progress', leave=False):
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model.module.generate(inputs, max_length=MAX_TARGET_LENGTH, num_beams=4)
        # convert token IDs to strings
        predicted_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        target_texts = tokenizer.batch_decode(targets, skip_special_tokens=True)
        # append predicted and target texts for BLEU evaluation
        predictions.extend(predicted_texts)
        references.extend(target_texts)

In [35]:
# calculate BLEU scores
#bleu = corpus_bleu(predictions, [references])
 
bleu = corpus_bleu(predictions, references)

print(f"BLEU score: {bleu.score}")

BLEU score: 0.3704640151983367


In [36]:
from datasets import load_metric

metric = load_metric('sacrebleu')

assert len(predictions) == len(references), "The number of predictions and references should be the same."

list_references = []
for reference in references:
    list_references.append([reference])

# Note the change in the compute line
score = metric.compute(predictions=predictions, references=list_references)

print(f"SacreBLEU score: {score['score']}")


SacreBLEU score: 36.49161740730786


In [37]:
from sacrebleu import corpus_chrf
# Calculate CHR F++ scores
chrf = corpus_chrf(predictions, [references])
print(f"CHR F++ score: {chrf.score}")

CHR F++ score: 63.939258929880026


In [27]:
!pip install -U nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.


In [49]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('wordnet')

# Assumptions:
# predictions is a list of predicted sentences
# references is a list of reference sentences

assert len(predictions) == len(references), "The number of predictions and references should be the same."

scores = []

for prediction, reference in zip(predictions, references):
    tokenized_prediction = word_tokenize(prediction)
    tokenized_reference = word_tokenize(reference)
    score = nltk.translate.meteor_score.meteor_score([tokenized_reference], tokenized_prediction)
    scores.append(score)

average_meteor_score = sum(scores) / len(scores)
print(f"Average METEOR score: {average_meteor_score}")


AttributeError: module 'sacrebleu' has no attribute 'sentence_meteor'

In [59]:
!pip install bert_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.4 MB/s eta 0:00:00


In [38]:
from datasets import load_metric
import numpy as np


metric = load_metric('bertscore')

assert len(predictions) == len(references), "The number of predictions and references should be the same."

# Compute the score
score = metric.compute(predictions=predictions, references=references, lang='en')

print(f"BERTScore: {np.mean(score['precision'])}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore: 0.9464624207909023


In [57]:
import datasets
datasets.list_metrics()

['accuracy',
 'bertscore',
 'bleu',
 'bleurt',
 'brier_score',
 'cer',
 'character',
 'charcut_mt',
 'chrf',
 'code_eval',
 'comet',
 'competition_math',
 'coval',
 'cuad',
 'exact_match',
 'f1',
 'frugalscore',
 'glue',
 'google_bleu',
 'indic_glue',
 'mae',
 'mahalanobis',
 'mape',
 'mase',
 'matthews_correlation',
 'mauve',
 'mean_iou',
 'meteor',
 'mse',
 'nist_mt',
 'pearsonr',
 'perplexity',
 'poseval',
 'precision',
 'r_squared',
 'recall',
 'rl_reliability',
 'roc_auc',
 'rouge',
 'sacrebleu',
 'sari',
 'seqeval',
 'smape',
 'spearmanr',
 'squad',
 'squad_v2',
 'super_glue',
 'ter',
 'trec_eval',
 'wer',
 'wiki_split',
 'xnli',
 'xtreme_s',
 'AlhitawiMohammed22/CER_Hu-Evaluation-Metrics',
 'BucketHeadP65/confusion_matrix',
 'BucketHeadP65/roc_curve',
 'Drunper/metrica_tesi',
 'Felipehonorato/eer',
 'He-Xingwei/sari_metric',
 'JP-SystemsX/nDCG',
 'Josh98/nl2bash_m',
 'Kyle1668/squad',
 'Muennighoff/code_eval',
 'NCSOFT/harim_plus',
 'Natooz/ece',
 'NikitaMartynov/spell-check-met

In [46]:
pip install pyter3


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [47]:
import pyter

# Assumptions:
# predictions is a list of predicted sentences
# references is a list of reference sentences

assert len(predictions) == len(references), "The number of predictions and references should be the same."

scores = []

for prediction, reference in zip(predictions, references):
    score = pyter.ter(reference.split(), prediction.split())
    scores.append(score)

average_ter_score = sum(scores) / len(scores)
print(f"Average TER score: {average_ter_score}")


Average TER score: 1.0390399552765


In [40]:
i=0
print(validation_dataset[i])
print('----')
print(predictions[i])
print('----')
print(references[i])

{'tripleset': [['Hawaii Five-O', 'NOTES', 'Episode: The Flight of the Jewels'], ['[TABLECONTEXT]', '[TITLE]', 'Jeff Daniels'], ['[TABLECONTEXT]', 'TITLE', 'Hawaii Five-O']], 'subtree_was_extended': True, 'annotations': {'source': ['WikiTableQuestions_lily'], 'text': ['Jeff Daniels played in the Hawaii Five-O episode The Flight of the Jewels']}}
----
Jeff Daniels aired the episode The Flight of the Jewels in Hawaii Five-O.
----
Jeff Daniels played in the Hawaii Five-O episode The Flight of the Jewels


In [41]:
i=10
print(validation_dataset[i])
print('----')
print(predictions[i])
print('----')
print(references[i])

{'tripleset': [["ricoh women's british open 1", 'WINNING_SCORE', '74-67-71-73=285'], ['2 aug 2009', 'TOURNAMENT', "ricoh women's british open 1"]], 'subtree_was_extended': True, 'annotations': {'source': ['WikiSQL_decl_sents'], 'text': ['The date of the winning score 74-67-71-73=285 is 2 aug 2009.']}}
----
The winning score of the tournament on 2 Aug 2009 was 74-67-71-73=285
----
The date of the winning score 74-67-71-73=285 is 2 aug 2009.


In [42]:
i=30
print(validation_dataset[i])
print('----')
print(predictions[i])
print('----')
print(references[i])

{'tripleset': [['the kissing game', 'DIRECTED_BY', 'patrick duffy']], 'subtree_was_extended': False, 'annotations': {'source': ['WikiSQL_decl_sents'], 'text': ['Patrick Duffy directed "The Kissing Game".']}}
----
Patrick duffy directed the episode titled "the kissing game".
----
Patrick Duffy directed "The Kissing Game".
